In [1]:
import requests
import json
import pandas as pd
import os
import time 
from IPython.display import clear_output 
from operator import itemgetter
import numpy as np
from pandas.io.json import json_normalize

#set wd
# os.chdir("//rca-file-server/RCA/Data Science/Australia Feeds/Company Matching Geocodes")



df = pd.read_excel("company_address2.xlsx")  
# df1 = pd.read_excel("this.xlsx")

/Users/randyclinton/.pyenv/versions/3.7.9/envs/ds/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  app.launch_new_instance()


In [2]:
# !pip install xlrd==1.2

In [3]:
df.shape
print(type(df['response'][1]))

<class 'str'>


In [4]:
df.head()

,Concatenated Address,response
0,"BUNDALL ROAD CNR SLATYER AVE 11L, BUNDALL ,Au...",{'address_components': [{'long_name': 'Bundall...
1,"CHARLES ST 22017 06, ST MARYS ,Australia",{'address_components': [{'long_name': 'St. Mar...
2,"HAMILTON AVE 22017 08, SURFERS PARADISE ,Austr...",{'address_components': [{'long_name': 'Hamilto...
3,"LOGAN RD 72016 19, MOUNT GRAVATT ,Australia",{'address_components': [{'long_name': 'Logan R...
4,"RAILWAY ST 32016 08, WENTWORTHVILLE ,Australia",{'address_components': [{'long_name': 'Railway...


In [5]:
from pprint import pprint

In [6]:
# json.loads(df.response[0])

In [7]:
df['response'] = df['response'].str.replace('True','true')
df['response'] = df['response'].str.replace('False','false')
df['response'] = df['response'].str.replace('None','"none"')

df['response'] = df['response'].replace("\\", r"\\")


In [8]:
type(json.loads(df['response'][1]))

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [106]:
json.loads(df.response[1])

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [20]:
# # rows list initialization 
# rows = [] 
  
# # appending rows 
# for data in list: 
#     data_row = df['response']  
      
#     for row in data_row: 
#         row['Name']= json.loads(data_row) 
#         rows.append(row) 
  
# # using data frame 
# df = pd.DataFrame(rows) 
  
# # print(df) 

In [9]:
df_head = df.head()

In [10]:
df_head['response_json'] = df_head['response'].apply(json.loads)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [11]:
# df['response'] = df['response'].str.replace('"', "'")
# df['response'] = df['response'].str.replace("'", '"')

In [12]:
def loads_catch(row):
    try:
        json.loads(row)
    except Exception as e:
        pprint(row)
        print(e)

In [13]:
[{'long_name': 'Bundall Road & Slatyer Avenue', " "'short_name': 'Bundal

SyntaxError: EOL while scanning string literal (<ipython-input-13-83ae16b39c86>, line 1)

In [ ]:
" '"none",

In [14]:
df.loc[0:5,'response'].apply(loads_catch)

("{'address_components': [{'long_name': 'Bundall Road & Slatyer Avenue', "
 "'short_name': 'Bundall Rd & Slatyer Ave', 'types': ['intersection']}, "
 "{'long_name': 'Bundall', 'short_name': 'Bundall', 'types': ['locality', "
 "'political']}, {'long_name': 'City of Gold Coast', 'short_name': 'Gold "
 "Coast', 'types': ['administrative_area_level_2', 'political']}, "
 "{'long_name': 'Queensland', 'short_name': 'QLD', 'types': "
 "['administrative_area_level_1', 'political']}, {'long_name': 'Australia', "
 "'short_name': 'AU', 'types': ['country', 'political']}, {'long_name': "
 "'4217', 'short_name': '4217', 'types': ['postal_code']}], "
 "'formatted_address': 'Bundall Rd & Slatyer Ave, Bundall QLD 4217, "
 "Australia', 'geometry': {'location': {'lat': -27.9976302, 'lng': "
 "153.4141172}, 'location_type': 'GEOMETRIC_CENTER', 'viewport': {'northeast': "
 "{'lat': -27.9962812197085, 'lng': 153.4154661802915}, 'southwest': {'lat': "
 "-27.9989791802915, 'lng': 153.4127682197085}}}, 'partia

0    None
1    None
2    None
3    None
4    None
5    None
Name: response, dtype: object

In [60]:
def create_output_df():
    df_dict = df_head.response_json.to_dict()
    list_of_results = []
    for col in df_dict:
        new_df = pd.json_normalize(df_dict[col])
        list_of_results.append(new_df)
    final_df = pd.concat(list_of_results)
    return final_df

In [62]:
newDF = create_output_df()
newDF.head()

,address_components,formatted_address,partial_match,place_id,types,geometry.location.lat,geometry.location.lng,geometry.location_type,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,...,reverse_geocode.adminLevel3_id,reverse_geocode.adminLevel3_tx,reverse_geocode.adminLevel3Type_tx,reverse_geocode.adminLevel3_display_city_name_fg,reverse_geocode.indexMarket_id,reverse_geocode.cbsa_id,reverse_geocode.csa_id,reverse_geocode.neighborhood_id,reverse_geocode.neighborhood_tx,reverse_geocode.neighborhood_display_city_name_fg
0,[{'long_name': 'Bundall Road & Slatyer Avenue'...,"Bundall Rd & Slatyer Ave, Bundall QLD 4217, Au...",True,EjVCdW5kYWxsIFJkICYgU2xhdHllciBBdmUsIEJ1bmRhbG...,[intersection],-27.997630,153.414117,GEOMETRIC_CENTER,-27.996281,153.415466,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"[{'long_name': 'St. Marys', 'short_name': 'St ...","St Marys, GA 31558, USA",True,ChIJiVXa-J4D5YgR_9oA5sRZda4,"[locality, political]",30.747895,-81.570611,APPROXIMATE,30.844184,-81.505520,...,31602.0,St. Marys,County_Subdivision,False,32.0,86.0,70.0,NaN,NaN,NaN
0,"[{'long_name': 'Hamilton Avenue', 'short_name'...","Hamilton Ave, Surfers Paradise QLD 4217, Austr...",NaN,EjJIYW1pbHRvbiBBdmUsIFN1cmZlcnMgUGFyYWRpc2UgUU...,[route],-28.006737,153.429215,GEOMETRIC_CENTER,-28.005474,153.431928,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"[{'long_name': 'Logan Road', 'short_name': 'Lo...","Logan Rd, Mount Gravatt QLD 4122, Australia",NaN,EitMb2dhbiBSZCwgTW91bnQgR3JhdmF0dCBRTEQgNDEyMi...,[route],-27.537256,153.078453,GEOMETRIC_CENTER,-27.528479,153.081776,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,"[{'long_name': 'Railway Street', 'short_name':...","Railway St, Wentworthville NSW 2145, Australia",NaN,ChIJF3Mpf5SiEmsR_vq9IgBsZZ0,[route],-33.803699,150.974037,GEOMETRIC_CENTER,-33.800599,150.975829,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9647.0,"Parramatta, Sydney",True


In [ ]:
s2 = df_head.to_dict()

In [35]:
sample_dict = json.loads(df_head.response[2])

In [40]:
pd.json_normalize(s2)

KeyError: '0'

In [21]:
final = pd.DataFrame.from_dict(pd.json_normalize(df1['response']) ,orient='columns')


NameError: name 'df1' is not defined

In [448]:
json.loads(df['response'][0])

{'address_components': [{'long_name': 'Bundall Road & Slatyer Avenue',
   'short_name': 'Bundall Rd & Slatyer Ave',
   'types': ['intersection']},
  {'long_name': 'Bundall',
   'short_name': 'Bundall',
   'types': ['locality', 'political']},
  {'long_name': 'City of Gold Coast',
   'short_name': 'Gold Coast',
   'types': ['administrative_area_level_2', 'political']},
  {'long_name': 'Queensland',
   'short_name': 'QLD',
   'types': ['administrative_area_level_1', 'political']},
  {'long_name': 'Australia',
   'short_name': 'AU',
   'types': ['country', 'political']},
  {'long_name': '4217', 'short_name': '4217', 'types': ['postal_code']}],
 'formatted_address': 'Bundall Rd & Slatyer Ave, Bundall QLD 4217, Australia',
 'geometry': {'location': {'lat': -27.9976302, 'lng': 153.4141172},
  'location_type': 'GEOMETRIC_CENTER',
  'viewport': {'northeast': {'lat': -27.9962812197085,
    'lng': 153.4154661802915},
   'southwest': {'lat': -27.9989791802915, 'lng': 153.4127682197085}}},
 'partia

In [449]:
df['response'][0]

'{"address_components": [{"long_name": "Bundall Road & Slatyer Avenue", "short_name": "Bundall Rd & Slatyer Ave", "types": ["intersection"]}, {"long_name": "Bundall", "short_name": "Bundall", "types": ["locality", "political"]}, {"long_name": "City of Gold Coast", "short_name": "Gold Coast", "types": ["administrative_area_level_2", "political"]}, {"long_name": "Queensland", "short_name": "QLD", "types": ["administrative_area_level_1", "political"]}, {"long_name": "Australia", "short_name": "AU", "types": ["country", "political"]}, {"long_name": "4217", "short_name": "4217", "types": ["postal_code"]}], "formatted_address": "Bundall Rd & Slatyer Ave, Bundall QLD 4217, Australia", "geometry": {"location": {"lat": -27.9976302, "lng": 153.4141172}, "location_type": "GEOMETRIC_CENTER", "viewport": {"northeast": {"lat": -27.9962812197085, "lng": 153.4154661802915}, "southwest": {"lat": -27.9989791802915, "lng": 153.4127682197085}}}, "partial_match": true, "place_id": "EjVCdW5kYWxsIFJkICYgU2xh

In [ ]:
for address in list_of_address:
    geocoded_response = geocode(address)
    save_to_file(geocode_response)
    
save_to_file(key: str, response: Dict) -> None:
    output = {key: response}
    with file(FILE_NAME, 'w+') as f:
        f.write(output)
    